In [6]:
# making the dataset
from sklearn.datasets import make_moons
X,y = make_moons(n_samples=10000, noise=0.4, random_state=42)

In [7]:
# using train test split to split the database
from sklearn.model_selection import train_test_split

X_train,X_test, y_train, y_test = train_test_split(X, y,test_size=0.35, random_state=42)

In [11]:
# defining the model
from sklearn.tree import DecisionTreeClassifier

d_tree = DecisionTreeClassifier(max_depth=2)

In [12]:
# using gridsearch
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

gsc = GridSearchCV(
        estimator= d_tree,
        param_grid={
             'max_leaf_nodes': [2,4,6,8,10]
        },
        cv=5, scoring='neg_mean_squared_error', verbose=1,n_jobs=-1
        
    )

In [16]:
#now finding the best params
grid_result = gsc.fit(X_test,y_test)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.0s finished


In [19]:
best_params = grid_result.best_params_
print(best_params)

{'max_leaf_nodes': 4}


In [20]:
best_tree = DecisionTreeClassifier(max_depth=2,max_leaf_nodes=4)

In [21]:
best_tree.fit(X_test, y_test)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
                       max_features=None, max_leaf_nodes=4,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [22]:
# testing the accuracy
from sklearn.metrics import accuracy_score

y_pred = gsc.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.8602857142857143


### actual way

In [25]:
params = {'max_leaf_nodes':list(range(2,100)), "min_samples_split":[2,3,4]}
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1, verbose=0, cv=3)
grid_search_cv.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=42,
                                              splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 

In [26]:
grid_search_cv.best_estimator_

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=30,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [27]:
from sklearn.metrics import accuracy_score

y_pred = grid_search_cv.predict(X_test)
accuracy_score(y_test, y_pred)

0.8642857142857143

### growing a forest

In [28]:
# splitting training data into subsets
from sklearn.model_selection import ShuffleSplit

n_trees =1000
n_instances =100

mini_sets = []
rs = ShuffleSplit(n_splits=n_trees, test_size=n_instances, random_state=42)

for mini_train_index,mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train,y_mini_train))
    

In [31]:
from sklearn.base import clone
import numpy as np

forest = [clone(grid_search_cv.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test,y_pred))
    
np.mean(accuracy_scores)

0.8618965714285715

In [33]:
# keeping only the majority id tge predictions using mdoe

Y_pred = np.empty([n_trees, len(X_test)],dtype=np.uint8)
for tree_index, tree in enumerate(forest):
    Y_pred[tree_index]= tree.predict(X_test)

In [34]:
from scipy.stats import mode
y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

In [35]:
# evaluating these prediction on a test set
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.8648571428571429